<a href="https://colab.research.google.com/github/gwagdalf/colab2502/blob/main/chapter_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10장 임베딩 모델로 데이터 의미 압축하기

colab 기설치 완료

In [1]:
!python3 -V
# Python 3.11.11
!pip show transformers
# Name: transformers Version: 4.48.3
!pip show accelerate
# Name: accelerate Version: 1.3.0
!pip show huggingface_hub
#Name: huggingface-hub Version: 0.28.1
!pip show gcsfs
# Name: gcsfs Version: 2024.10.0
!pip show fsspec
# Name: fsspec Version: 2024.10.0
!pip show openai
# Name: openai Version: 1.61.1
!nvcc --version
# 12.5, nvcc: NVIDIA (R) Cuda compiler driver Cuda compilation tools, release 12.5, V12.5.82 Build cuda_12.5.r12.5/compiler.34385749_0
!pip show torch
# Name: torch Version: 2.5.1+cu124

Python 3.11.11
Name: transformers
Version: 4.48.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers
Name: accelerate
Version: 1.3.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft
Name: huggingface-hub
Version: 0.28.1
Summary: Client lib

In [ ]:
#!pip install transformers==4.40.1 datasets==2.19.0 sentence-transformers==2.7.0 faiss-cpu==1.8.0 llama-index==0.10.34 llama-index-embeddings-huggingface==0.2.0 -qqq

최신버전으로 module 추가설치

In [1]:
!pip install datasets sentence-transformers faiss-cpu llama-index llama-index-embeddings-huggingface -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

In [3]:
!pip show datasets
# Name: datasets Version: 3.3.2
!pip show sentence-transformers
# Name: sentence-transformers Version: 3.4.1
!pip show faiss-cpu
# Name: faiss-cpu Version: 1.10.0
!pip show llama-index
# Name: llama-index Version: 0.12.23
!pip show llama-index-embeddings-huggingface
# Name: llama-index-embeddings-huggingface Version: 0.5.2

Name: datasets
Version: 3.3.2
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: 
Name: sentence-transformers
Version: 3.4.1
Summary: State-of-the-Art Text Embeddings
Home-page: https://www.SBERT.net
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, Pillow, scikit-learn, scipy, torch, tqdm, transformers
Required-by: llama-index-embeddings-huggingface
Name: faiss-cpu
Version: 1.10.0
Summary: A library for efficient similarity search and clustering of dense vectors.
Home-page: 
Author: 
Author-ema

## 예제 10.1 문장 임베딩을 활용한 단어 간 유사도 계산

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

smodel = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
dense_embeddings = smodel.encode(['학교', '공부', '운동'])
cosine_similarity(dense_embeddings) # 코사인 유사도

# array(
#     학교           공부        운동
#학교 [[1.0000001 , 0.5950744 , 0.32537547],
#공부 [0.5950744 , 1.0000002 , 0.54595673],
#운동 [0.32537547, 0.54595673, 0.99999976]], dtype=float32)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

array([[0.9999999 , 0.5950743 , 0.3253754 ],
       [0.5950743 , 1.        , 0.54595685],
       [0.3253754 , 0.54595685, 1.        ]], dtype=float32)

## 예제 10.2 원핫 인코딩의 한계
원-핫 인코딩.

school: [1, 0, 0].

study: [0, 1, 0].

workout: [0, 0, 1].

각 단어는 벡터로 표현되며, 벡터의 크기는 단어 목록의 크기와 같습니다.  

각 벡터에서 단 하나의 요소만 1이고, 나머지는 모두 0입니다.  

여기서 1의 위치는 해당 단어가 단어 목록에서 몇 번째에 위치하는지를 나타냅니다.  

마치 단어 목록에서 해당 단어 위치만 "뜨겁게(hot)" 켜져 있는 것처럼 보여서 "원-핫(one-hot)"   인코딩이라고 불립니다.

In [3]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

word_dict = {"school": np.array([[1, 0, 0]]),
"study": np.array([[0, 1, 0]]),
"workout": np.array([[0, 0, 1]])
}

# 두 단어 사이의 코사인 유사도 계산하기
cosine_school_study = cosine_similarity(word_dict["school"], word_dict['study']) # 0
print(f'school-study:{cosine_school_study}')
cosine_school_workout = cosine_similarity(word_dict['school'], word_dict['workout']) # 0
print(f'school-workout:{cosine_school_workout}')

school-study:[[0.]]
school-workout:[[0.]]


원 핫 인코딩에서는 단어간 cosine similarity 가 0임  

즉 단어 간 의미 유사성을 전혀 반영하지 못합니다.

## 예제 10.3 Sentence-Transformers 라이브러리로 바이 인코더 생성하기
BERT : Bidirectional Encoder Representations from Transformers

In [4]:
from sentence_transformers import SentenceTransformer, models
# 사용할 BERT 모델
word_embedding_model = models.Transformer('klue/roberta-base')
# 풀링 층 차원 입력하기
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# 두 모듈 결합하기
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

print("모델 구조:")
print(model)
# SentenceTransformer(
#   (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel
#   (1): Pooling({
#     "word_embedding_dimension":768,
#     "pooling_mode_cls_token":false,
#     "pooling_mode_mean_tokens":true,
#     "pooling_mode_max_tokens":false,
#     "pooling_mode_mean_sqrt_len_tokens":false,
#     "pooling_mode_weightedmean_tokens":false,
#     "pooling_mode_lasttoken":false,
#     "include_prompt":true
#     })
# )

print("word_embedding_model 의 get_word_embedding_dimension():", word_embedding_model.get_word_embedding_dimension())
print("Pooling model의 word_embedding_dimension1:", pooling_model.word_embedding_dimension)

print("\n모델 타입:", type(model))
# 모델 타입: <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

모델 구조:
SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)
word_embedding_model 의 get_word_embedding_dimension(): 768
Pooling model의 word_embedding_dimension1: 768

모델 타입: <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>


BERT 모델의 출력을 풀링 층을 통해 고정된 크기의 문장 임베딩으로 만든다

1.   pooling_model_cls_tokens : 첫 번째 토큰인 CLS 사용
2.   pooling_model_mean_tokens : 모든 토큰의 평균값 사용(기본값)
3.   pooling_model_max_tokens : 모든 토큰의 최대값 사용

## 예제 10.4 코드로 살펴보는 평균 모드


In [5]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

mean_pooling 사용 예

In [6]:
from sentence_transformers import SentenceTransformer, models
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 원본 문장 정의
sentences = [
    "나는 회사에 갑니다.",
    "나는 회사의 Team study 에서 LLM을 공부합니다.",
    "태준이는 스위스로 여행을 다녀왔습니다"
]

# 3가지 풀링 전략 모델 생성
def create_model(pooling_mode):
    word_embedding = models.Transformer('klue/roberta-base')
    pooling = models.Pooling(
        word_embedding.get_word_embedding_dimension(),
        pooling_mode=pooling_mode
    )
    return SentenceTransformer(modules=[word_embedding, pooling])

# 각 풀링 전략별 임베딩 생성
pooling_strategies = {
    'cls_token': create_model('cls'),
    'mean_tokens': create_model('mean'),
    'max_tokens': create_model('max')
}

# 결과 저장용 딕셔너리
results = {}

for strategy_name, model in pooling_strategies.items():
    embeddings = model.encode(sentences)
    sim_matrix = cosine_similarity(embeddings)
    results[strategy_name] = pd.DataFrame(
        sim_matrix,
        index=sentences,
        columns=sentences
    )

# 결과 출력
for strategy, df in results.items():
    print(f"\n=== {strategy.upper()} 풀링 전략 유사도 행렬 ===")
    print(df.round(3))

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== CLS_TOKEN 풀링 전략 유사도 행렬 ===
                                  나는 회사에 갑니다.  \
나는 회사에 갑니다.                             1.000   
나는 회사의 Team study 에서 LLM을 공부합니다.        0.996   
태준이는 스위스로 여행을 다녀왔습니다                    0.995   

                                  나는 회사의 Team study 에서 LLM을 공부합니다.  \
나는 회사에 갑니다.                                                  0.996   
나는 회사의 Team study 에서 LLM을 공부합니다.                             1.000   
태준이는 스위스로 여행을 다녀왔습니다                                         0.994   

                                  태준이는 스위스로 여행을 다녀왔습니다  
나는 회사에 갑니다.                                      0.995  
나는 회사의 Team study 에서 LLM을 공부합니다.                 0.994  
태준이는 스위스로 여행을 다녀왔습니다                             1.000  

=== MEAN_TOKENS 풀링 전략 유사도 행렬 ===
                                  나는 회사에 갑니다.  \
나는 회사에 갑니다.                             1.000   
나는 회사의 Team study 에서 LLM을 공부합니다.        0.772   
태준이는 스위스로 여행을 다녀왔습니다                    0.758   

                               

결과
https://docs.google.com/spreadsheets/d/1LhWUD_0pkXjdGlh422ujhD7Qhe-6EfDjewzGRNkOKBA/edit?usp=sharing

![Image](https://github.com/user-attachments/assets/fed028c5-6f91-4734-8f76-c5ee6610e122)

## 예제 10.5 코드로 살펴보는 최대 모드
3) BERT 모델에서 모든 일력 토큰의 출력 임베딩의 최대값을 문장 임베딩으로 사용한다

In [7]:
def max_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    token_embeddings[input_mask_expanded == 0] = -1e9
    return torch.max(token_embeddings, 1)[0]

## 예제 10.6 한국어 문장 임베딩 모델로 입력 문장 사이의 유사도 계산

In [8]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

embs = model.encode(['잠이 안 옵니다',
                     '졸음이 옵니다',
                     '기차가 옵니다'])

cos_scores = util.cos_sim(embs, embs)
print(cos_scores)
# tensor([[1.0000, 0.6410, 0.1887],
#         [0.6410, 1.0000, 0.2730],
#         [0.1887, 0.2730, 1.0000]])

tensor([[1.0000, 0.6410, 0.1887],
        [0.6410, 1.0000, 0.2730],
        [0.1887, 0.2730, 1.0000]])


![Image](https://github.com/user-attachments/assets/2d59d2c1-9871-4bb2-9988-e7a22103f3e8)

## 예제 10.7 CLIP 모델을 활용한 이미지와 텍스트 임베딩 유사도 계산

In [11]:
from PIL import Image
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('clip-ViT-B-32')

img_embs = model.encode([Image.open('dog.jpg'), Image.open('cat.jpg')])
text_embs = model.encode(['A dog on grass', 'Brown cat on yellow background'])

cos_scores = util.cos_sim(img_embs, text_embs)
print(cos_scores)
# tensor(
#[[0.2771, 0.1509],
#[0.2071, 0.3180]])

tensor([[0.2771, 0.1509],
        [0.2071, 0.3180]])


![Image](https://github.com/user-attachments/assets/b9028f61-606d-4bea-a825-e213ec23b464)

## 예제 10.8 실습에 사용할 모델과 데이터셋 불러오기

In [12]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

## 예제 10.9 실습 데이터에서 1,000개만 선택하고 문장 임베딩으로 변환
https://huggingface.co/datasets/klue/klue/viewer/mrc

In [13]:
klue_mrc_dataset_train = klue_mrc_dataset.train_test_split(train_size=1000, shuffle=False)['train']
embeddings = sentence_model.encode(klue_mrc_dataset_train['context'])
embeddings.shape
print(f"embeddings.shape: {embeddings.shape}")
print(embeddings)

# embeddings.shape: (1000, 768)
# [[ 0.5222375  -1.1391711   0.12654702 ... -1.1193508   0.03521788
#   -0.63249075]
#  [-0.36406726 -0.63196135  0.190007   ... -0.21136642  0.40211734
#    0.38462615]
#  [-0.36406726 -0.63196135  0.190007   ... -0.21136642  0.40211734
#    0.38462615]
#  ...
#  [-0.9290558  -0.7262963  -0.37647069 ... -0.05437075  0.72452474
#   -0.45626393]
#  [-0.943096   -1.0330368   0.25581586 ... -0.24693933  0.28673905
#   -0.35674644]
#  [-0.8397218  -0.6824467  -0.07826263 ... -0.310852   -0.05979971
#   -0.75422746]]

embeddings.shape: (1000, 768)
[[ 0.5222372  -1.1391715   0.12654728 ... -1.1193498   0.03521844
  -0.6324905 ]
 [-0.36406684 -0.6319616   0.19000682 ... -0.21136579  0.4021174
   0.38462663]
 [-0.36406684 -0.6319616   0.19000682 ... -0.21136579  0.4021174
   0.38462663]
 ...
 [-0.92905575 -0.72629637 -0.37646997 ... -0.05437021  0.72452426
  -0.45626414]
 [-0.9430958  -1.0330368   0.25581682 ... -0.24693918  0.28673935
  -0.35674646]
 [-0.8397217  -0.68244684 -0.07826207 ... -0.31085145 -0.05979928
  -0.7542277 ]]


## 예제 10.10 KNN 검색 인덱스를 생성하고 문장 임베딩 저장
Faiss 모듈에 대한 설명 (Python)  

Faiss (Facebook AI Similarity Search)는 대규모 고차원 벡터의 효율적인 유사성 검색 및 클러스터링을 위한 라이브러리입니다.  

Facebook AI Research (FAIR) 팀에서 개발했으며, 특히 벡터 데이터베이스를 구축하거나 유사 이미지 검색, 추천 시스템 등에서 빠르고 효율적인 검색이 필요한 경우에 유용하게 사용됩니다.

In [14]:
import faiss
# 인덱스 만들기
index = faiss.IndexFlatL2(embeddings.shape[1])
# 인덱스에 임베딩 저장하기
index.add(embeddings)
print(index)
#<faiss.swigfaiss_avx512.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7db08361d590> >

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7a1954286430> >


## 예제 10.11 의미 검색의 장점

0번, title = 제주도 장마 시작 … 중부는 이달 말부터, context = 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다....

In [15]:
query = "이번 연도에는 언제 비가 많이 올까?"
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

# for idx in indices[0]:
#   print(klue_mrc_dataset['context'][idx][:50])

for i, idx in enumerate(indices[0]):
    print(f"Score: {distances[0][i]:.2f} / Context: {klue_mrc_dataset['context'][idx][:50]}") # score 와 context

# 출력 결과
# (정답) Score: 300.92 / Context: 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은
# (오답) Score: 420.86 / Context: 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그
# (오답) Score: 420.86 / Context: 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그

Score: 300.92 / Context: 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 
Score: 420.86 / Context: 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그
Score: 420.86 / Context: 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그


## 예제 10.12 의미 검색의 한계

https://huggingface.co/datasets/klue/klue/viewer/mrc  

https://huggingface.co/datasets/klue/klue/viewer/mrc?views%5B%5D=mrc_train&row=3

In [16]:
query = klue_mrc_dataset[3]['question'] # 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

# for idx in indices[0]:
#   print(klue_mrc_dataset['context'][idx][:50])

for i, idx in enumerate(indices[0]):
    print(f"Score: {distances[0][i]:.2f} / Context: {klue_mrc_dataset['context'][idx][:50]}") # score 와 context

print(f"index: {index}")
print(f"indices: {indices}")
print(f"indices[0]: {indices[0]}")


# Score: 240.37 / Context: 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로
# Score: 240.37 / Context: 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로
# Score: 242.09 / Context: 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스
# index: <faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7a1954286430> >
# indices: [[78 79  3]]
# indices[0]: [78 79  3]

Score: 240.37 / Context: 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 
Score: 240.37 / Context: 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 
Score: 242.09 / Context: 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스
index: <faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7a1954286430> >
indices: [[78 79  3]]
indices[0]: [78 79  3]


## 예제 10.13 라마인덱스에서 Sentence-Transformers 임베딩 모델 활용
service_context 가 deprecated 되어 settings로 대체

In [21]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")
Settings.embed_model = embed_model
# Settings.llm = None # llm 설정 제거. 필요시 Settings.llm = ... 으로 설정
# 로컬 모델 활용하기
# Settings.embed_model="local"

text_list = klue_mrc_dataset[:100]['context']
documents = [Document(text=t) for t in text_list]

index_llama = VectorStoreIndex.from_documents(
    documents,
    # service_context=service_context, # service_context 파라미터 제거
)

### 10.13.1 llama-index query 추가 예제

In [24]:
# ---------------------------------------
# 추가, 쿼리 예제
retriever_engine = index_llama.as_retriever(similarity_top_k=3)

# 질문 수행
question = "로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?"
results = retriever_engine.retrieve(question)

# 결과 출력
for idx, node in enumerate(results, 1):
    print(f"[결과 {idx}] Score: {node.score:.3f}")
    print(f"Context: {node.node.text[:100]}...\n{'-'*10}")
    #print(node)


# [결과 1] Score: 0.460
# Context: 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스터 대학교에서 핵물리학으로 박사 학위를 마쳤다. 제2차 세계대전중에 그는 매사추세츠 공과대...
# ----------
# [결과 2] Score: 0.435
# Context: 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 하는 미 육군 주체의 연합군 남서 태평양 방면군은 1944년 후반 마침내 필리핀을 진공하기...
# ----------


[결과 1] Score: 0.460
Context: 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스터 대학교에서 핵물리학으로 박사 학위를 마쳤다. 제2차 세계대전중에 그는 매사추세츠 공과대...
----------
[결과 2] Score: 0.435
Context: 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 하는 미 육군 주체의 연합군 남서 태평양 방면군은 1944년 후반 마침내 필리핀을 진공하기...
----------


In [29]:
import os
from openai import OpenAI
from google.colab import userdata

# OpenAI API 키를 userdata에서 가져오기
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# 쿼리 엔진 생성
query_engine = index_llama.as_query_engine()

# 질문
query_text = "로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?"

# 쿼리 수행
response = query_engine.query(query_text)

# 결과 출력 (score, context)
for node in response.source_nodes:
    print(f"Score: {node.score:.3f}")
    print(f"Context: {node.node.text[:100]}...\n{'-'*10}")

# Score: 0.460
# Context: 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스터 대학교에서 핵물리학으로 박사 학위를 마쳤다. 제2차 세계대전중에 그는 매사추세츠 공과대...
# ----------
# Score: 0.435
# Context: 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 하는 미 육군 주체의 연합군 남서 태평양 방면군은 1944년 후반 마침내 필리핀을 진공하기...
# ----------


Score: 0.460
Context: 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스터 대학교에서 핵물리학으로 박사 학위를 마쳤다. 제2차 세계대전중에 그는 매사추세츠 공과대...
----------
Score: 0.435
Context: 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 하는 미 육군 주체의 연합군 남서 태평양 방면군은 1944년 후반 마침내 필리핀을 진공하기...
----------


In [32]:
# ServiceContext deprecated 되어 주석 처리
# from llama_index.core import VectorStoreIndex, ServiceContext
# from llama_index.core import Document
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# embed_model = HuggingFaceEmbedding(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")
# service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)
# # 로컬 모델 활용하기
# # service_context = ServiceContext.from_defaults(embed_model="local")

# text_list = klue_mrc_dataset[:100]['context']
# documents = [Document(text=t) for t in text_list]

# index_llama = VectorStoreIndex.from_documents(
#     documents,
#     service_context=service_context,
# )

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ValueError: ServiceContext is deprecated. Use llama_index.settings.Settings instead, or pass in modules to local functions/methods/interfaces.
See the docs for updated usage/migration: 
https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/

## 예제 10.14 BM25 클래스 구현

In [18]:
import math
import numpy as np
from typing import List
from transformers import PreTrainedTokenizer
from collections import defaultdict

class BM25:
  def __init__(self, corpus:List[List[str]], tokenizer:PreTrainedTokenizer):
    self.tokenizer = tokenizer
    self.corpus = corpus
    self.tokenized_corpus = self.tokenizer(corpus, add_special_tokens=False)['input_ids']
    self.n_docs = len(self.tokenized_corpus)
    self.avg_doc_lens = sum(len(lst) for lst in self.tokenized_corpus) / len(self.tokenized_corpus)
    self.idf = self._calculate_idf()
    self.term_freqs = self._calculate_term_freqs()

  def _calculate_idf(self):
    idf = defaultdict(float)
    for doc in self.tokenized_corpus:
      for token_id in set(doc):
        idf[token_id] += 1
    for token_id, doc_frequency in idf.items():
      idf[token_id] = math.log(((self.n_docs - doc_frequency + 0.5) / (doc_frequency + 0.5)) + 1)
    return idf

  def _calculate_term_freqs(self):
    term_freqs = [defaultdict(int) for _ in range(self.n_docs)]
    for i, doc in enumerate(self.tokenized_corpus):
      for token_id in doc:
        term_freqs[i][token_id] += 1
    return term_freqs

  def get_scores(self, query:str, k1:float = 1.2, b:float=0.75):
    query = self.tokenizer([query], add_special_tokens=False)['input_ids'][0]
    scores = np.zeros(self.n_docs)
    for q in query:
      idf = self.idf[q]
      for i, term_freq in enumerate(self.term_freqs):
        q_frequency = term_freq[q]
        doc_len = len(self.tokenized_corpus[i])
        score_q = idf * (q_frequency * (k1 + 1)) / ((q_frequency) + k1 * (1 - b + b * (doc_len / self.avg_doc_lens)))
        scores[i] += score_q
    return scores

  def get_top_k(self, query:str, k:int):
    scores = self.get_scores(query)
    top_k_indices = np.argsort(scores)[-k:][::-1]
    top_k_scores = scores[top_k_indices]
    return top_k_scores, top_k_indices

## 예제 10.15 BM25 점수 계산 확인해 보기

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

bm25 = BM25(['안녕하세요', '반갑습니다', '안녕 서울'], tokenizer)
bm25.get_scores('안녕')
# array([0.44713859, 0.        , 0.52354835])

## 예제 10.16 BM25 검색 결과의 한계

In [ ]:
# BM25 검색 준비
bm25 = BM25(klue_mrc_dataset['context'], tokenizer)

query = "이번 연도에는 언제 비가 많이 올까?"
_, bm25_search_ranking = bm25.get_top_k(query, 100)

for idx in bm25_search_ranking[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

# 출력 결과
# 갤럭시S5 언제 발매한다는 건지언제는 “27일 판매한다”고 했다가 “이르면 26일 판매한다 (오답)
# 인구 비율당 노벨상을 세계에서 가장 많이 받은 나라, 과학 논문을 가장 많이 쓰고 의료 특 (오답)
# 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은  (정답)

## 예제 10.17 BM25 검색 결과의 장점

In [ ]:
query = klue_mrc_dataset[3]['question']  # 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
_, bm25_search_ranking = bm25.get_top_k(query, 100)

for idx in bm25_search_ranking[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

# 출력 결과
# 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스 (정답)
# ;메카동(メカドン)                                                      (오답)
# :성우 : 나라하시 미키(ならはしみき)
# 길가에 버려져 있던 낡은 느티나
# ;메카동(メカドン)                                                      (오답)
# :성우 : 나라하시 미키(ならはしみき)
# 길가에 버려져 있던 낡은 느티나

## 예제 10.18 상호 순위 조합 함수 구현

In [ ]:
from collections import defaultdict

def reciprocal_rank_fusion(rankings:List[List[int]], k=5):
    rrf = defaultdict(float)
    for ranking in rankings:
        for i, doc_id in enumerate(ranking, 1):
            rrf[doc_id] += 1.0 / (k + i)
    return sorted(rrf.items(), key=lambda x: x[1], reverse=True)

## 예제 10.19 예시 데이터에 대한 상호 순위 조합 결과 확인하기

In [ ]:
rankings = [[1, 4, 3, 5, 6], [2, 1, 3, 6, 4]]
reciprocal_rank_fusion(rankings)

# [(1, 0.30952380952380953),
#  (3, 0.25),
#  (4, 0.24285714285714285),
#  (6, 0.2111111111111111),
#  (2, 0.16666666666666666),
#  (5, 0.1111111111111111)]

## 예제 10.20 하이브리드 검색 구현하기

In [ ]:
def dense_vector_search(query:str, k:int):
  query_embedding = sentence_model.encode([query])
  distances, indices = index.search(query_embedding, k)
  return distances[0], indices[0]

def hybrid_search(query, k=20):
  _, dense_search_ranking = dense_vector_search(query, 100)
  _, bm25_search_ranking = bm25.get_top_k(query, 100)

  results = reciprocal_rank_fusion([dense_search_ranking, bm25_search_ranking], k=k)
  return results

## 예제 10.21 예시 데이터에 대한 하이브리드 검색 결과 확인

In [ ]:
query = "이번 연도에는 언제 비가 많이 올까?"
print("검색 쿼리 문장: ", query)
results = hybrid_search(query)
for idx, score in results[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

print("=" * 80)
query = klue_mrc_dataset[3]['question'] # 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
print("검색 쿼리 문장: ", query)

results = hybrid_search(query)
for idx, score in results[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

# 출력 결과
# 검색 쿼리 문장:  이번 연도에는 언제 비가 많이 올까?
# 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은  (정답)
# 갤럭시S5 언제 발매한다는 건지언제는 “27일 판매한다”고 했다가 “이르면 26일 판매한다  (오답)
# 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그 (오답)
# ================================================================================
# 검색 쿼리 문장:  로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
# 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스 (정답)
# 1950년대 말 매사추세츠 공과대학교의 동아리 테크모델철도클럽에서 ‘해커’라는 용어가 처음 (오답)
# 1950년대 말 매사추세츠 공과대학교의 동아리 테크모델철도클럽에서 ‘해커’라는 용어가 처음 (오답)